In [1]:
''' define the instant class and some useful scripts related to them '''

class instant:

    def __init__(instant=int, zipcode=str, coordinates=dict, forecasts=list, observation=dict):
        {'instant': instant,
        'zipcode': zipcode,
         'coordinates': coordinates,
        'forecasts': forecasts,
        'observation': observation
        }
    
    def count(instant):
        ''' Count the number of elemnets in the forecasts array '''
        
        return len(instant['forecats'])
        

def count(instant):
    ''' Count the number of elemnets in the forecasts array '''

    return len(instant['forecasts'])

def cast_count_all(col):
    ''' get a tally for the forecast counts per document 
    
    :param col: the collection you want to evaluate
    :type col: pymongo.collection.Collection
    '''
    print('getting started with cast_count_all')
    n = 0
    collection_cast_counts = {}
    # Go through each doc in the collection and count the number of items in the forecasts array.
    # Add to the tally for that count.
    for doc in col.find({}):
        n = count(doc)
        if n in collection_cast_counts:
            collection_cast_counts[n] += 1
        else:
            collection_cast_counts[n] = 1
    return collection_cast_counts

def itslegit(instant):
    ''' check the instant's weathers array for count. if it is 40, then the document is returned

    :param instant: the instant docuemnt to be legitimized
    :type instant: dictionary
    '''

    if count(instant) == 40:
        return True
    else:
        return False

def find_legit(collection):
    ''' find the 'legit' instants within the collection specified

    :param collection: database collection
    :type collection: pymongo.collection.Collection
    :return: list of documents
    '''

    return [item for item in collection.find({}) if itslegit(item)]

def load_legit(collection):
    ''' load the 'legit' instants to the remote database 

    :param collection: the collection you want to pull instants from
    :type collection: pymongo.collection.Collection
    '''
    from db_ops import Client, dbncol
    from config import uri, host, port

    legit_list = getlegit(collection)
    client = Client(host, port)
    col = dbncol(client, 'legit_inst', 'owmap')
    col.insert_many(legit_list)
    return

def test_load_legit(collection):
    ''' load the 'legit' instants to the remote database 

    :param collection: the collection you want to pull instants from
    :type collection: pymongo.collection.Collection
    '''
    from db_ops import Client, dbncol
    from config import host, port
    host = 'localhost'
    port = 27017
    legit_list = getlegit(collection)
    client = Client(host, port)
    col = dbncol(client, 'legit_inst', 'test')
    col.insert_many(legit_list)
    return

In [3]:
import db_ops
# from Extract.config import host, port
client = db_ops.Client(host=db_ops.host, port=db_ops.port)
collection = 'instant_temp'
database = 'owmap'
col = db_ops.dbncol(client, collection, database)

In [5]:
cast_count_all(col)

{1: 2160,
 2: 3240,
 3: 2160,
 4: 2160,
 5: 2160,
 6: 2160,
 7: 4320,
 8: 2160,
 9: 2160,
 10: 2160,
 11: 4319,
 12: 3168,
 13: 8883,
 14: 2286,
 15: 7237,
 16: 2298,
 17: 2982,
 18: 3149,
 19: 4703,
 20: 2923,
 22: 3012,
 23: 7587,
 25: 3659,
 26: 5024,
 28: 4115,
 29: 2944,
 30: 3067,
 31: 2864,
 32: 3437,
 33: 2492,
 34: 2043,
 21: 3813,
 24: 4701,
 27: 3230,
 35: 476,
 36: 315,
 37: 252,
 38: 59,
 39: 2}

In [4]:
inst_col = col.count_documents({})
print(inst_col)

119880


In [29]:
instant = col.find({})[0]
print(instant)
# import Extract as e

{'_id': ObjectId('5e9bb25c42f97b1723883d7c'), 'instant': 1587265200, 'zipcode': '27006', 'forecasts': [{'sunset_time': 0, 'sunrise_time': 0, 'clouds': 0, 'rain': {}, 'snow': {}, 'wind': {'speed': 1.7, 'deg': 39}, 'humidity': 55, 'pressure': {'press': 1019, 'sea_level': 1019}, 'temperature': {'temp': 280.13, 'temp_kf': -0.47, 'temp_max': 280.6, 'temp_min': 280.13}, 'status': 'Clear', 'detailed_status': 'clear sky', 'weather_code': 800, 'weather_icon_name': '01n', 'visibility_distance': None, 'dewpoint': None, 'humidex': None, 'heat_index': None, 'time_to_instant': 3178}], 'weather': {'sunset_time': 1587254263, 'sunrise_time': 1587206633, 'clouds': 0, 'rain': {}, 'snow': {}, 'wind': {'speed': 0.45, 'deg': 40, 'gust': 1.34}, 'humidity': 48, 'pressure': {'press': 1014, 'sea_level': None}, 'temperature': {'temp': 283.22, 'temp_kf': None, 'temp_max': 284.26, 'temp_min': 281.48}, 'status': 'Clear', 'detailed_status': 'clear sky', 'weather_code': 800, 'weather_icon_name': '01n', 'visibility_di

In [30]:
itslegit(instant)

False

In [31]:
legit_list = find_legit(col)

In [32]:
len(legit_list)

0

In [ ]:
load_legit(col)